In [1]:
import gc
import torch
from torch import nn
from torch.profiler import record_function, profile, ProfilerActivity, schedule
from einops import rearrange
from lrn_sparseatt.masks import BooleanMask, boolean_mask_to_jagged_indices
from lrn_sparseatt.ops import sparse_attn
from contextlib import contextmanager

N_RUNS = 50

@contextmanager
def profile_and_report(name):
    gc.collect()
    sched = schedule(skip_first=10, wait=5, warmup=5, active=N_RUNS - 20)
    try:
        with profile(activities=[ProfilerActivity.CPU], schedule=sched) as prof:
            with record_function(name):
                yield prof
    finally:
        print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=15))
        print(f"Total per run (us): {prof.key_averages().self_cpu_time_total / (N_RUNS-20):.2f}")

# Initialize scenario

First define the sizes of the inputs and of the model

In [2]:
seq_len = 128
d_model = 64
n_heads = 1
head_dim = d_model // n_heads

In [3]:
mask = BooleanMask.random(seq_len, 0.7).as_tensor(seq_len)
mask

tensor([[False, False,  True,  ..., False, False, False],
        [False,  True, False,  ...,  True, False, False],
        [ True,  True,  True,  ..., False, False,  True],
        ...,
        [False, False, False,  ...,  True, False, False],
        [ True,  True, False,  ..., False, False,  True],
        [False, False, False,  ...,  True, False, False]])

In [4]:
q = torch.randn((n_heads, seq_len, d_model // n_heads))
k = torch.randn((n_heads, seq_len, d_model // n_heads))
v = torch.randn((n_heads, seq_len, d_model // n_heads))

# Masked MHSA

In [5]:
def compute_attention(
    q: torch.Tensor,
    k: torch.Tensor,
    v: torch.Tensor,
    attn_mask: torch.Tensor,
) -> torch.Tensor:
    # q, k, v have shape [H, T, D]
    # attn_mask has shape [T, T] or [T, T]
    head_dim = q.size(2)

    attn_weights: torch.Tensor = torch.matmul(q, k.transpose(-2, -1)) / (
        head_dim**0.5
    )
    # attn_weights has shape [H, T, T]

    # attn_mask shape should be broadcastable to attn_weights shape
    attn_mask = attn_mask.unsqueeze(0)  # shape [1, T, T]
    attn_weights = attn_weights.masked_fill(~attn_mask, float("-inf"))
    attn_weights = torch.softmax(attn_weights, dim=-1)
    return torch.matmul(attn_weights, v)

In [6]:
with profile_and_report("dense_attention") as p:
    for _ in range(N_RUNS):
        res_dense = compute_attention(q, k, v, mask)
        p.step()

------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                    Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
           ProfilerStep*        20.12%     473.795us       100.00%       2.355ms      78.492us            30  
       aten::masked_fill         1.41%      33.193us        32.70%     770.089us      25.670us            30  
      aten::masked_fill_        27.61%     650.108us        27.61%     650.108us      21.670us            30  
            aten::matmul         3.27%      76.949us        23.64%     556.696us       9.278us            60  
           aten::softmax         0.69%      16.206us        14.85%     349.676us      11.656us            30  
          aten::_softmax        14.16%     333.470us        14.16%     333.470us      11.116us            30  
 

/Users/theodorebluche/src/learn-spare-attn/.venv/lib/python3.13/site-packages/torch/profiler/profiler.py:217: UserWarning: Warning: Profiler clears events at the end of each cycle.Only events from the current cycle will be reported.To keep events across cycles, set acc_events=True.
  _warn_once(


# Sparse mask

In [10]:
sp_mask = mask.to_sparse_csr().to(torch.float) * 0
sp_mask

tensor(crow_indices=tensor([   0,   38,   76,  114,  152,  190,  228,  266,
                             304,  342,  380,  418,  456,  494,  532,  570,
                             608,  646,  684,  722,  760,  798,  836,  874,
                             912,  950,  988, 1026, 1064, 1102, 1140, 1178,
                            1216, 1254, 1292, 1330, 1368, 1406, 1444, 1482,
                            1520, 1558, 1596, 1634, 1672, 1710, 1748, 1786,
                            1824, 1862, 1900, 1938, 1976, 2014, 2052, 2090,
                            2128, 2166, 2204, 2242, 2280, 2318, 2356, 2394,
                            2432, 2470, 2508, 2546, 2584, 2622, 2660, 2698,
                            2736, 2774, 2812, 2850, 2888, 2926, 2964, 3002,
                            3040, 3078, 3116, 3154, 3192, 3230, 3268, 3306,
                            3344, 3382, 3420, 3458, 3496, 3534, 3572, 3610,
                            3648, 3686, 3724, 3762, 3800, 3838, 3876, 3914,
            

In [19]:
sp_attn_weights = torch.sparse.sampled_addmm(sp_mask, q, k.transpose(-2, -1), beta=0.0, alpha=1.0 / (head_dim**0.5))
attn_weights = sp_attn_weights.to_dense()
attn_mask = mask.unsqueeze(0)  # shape [1, T, T]
attn_weights = attn_weights.masked_fill(~mask, float("-inf"))
attn_weights = torch.softmax(attn_weights, dim=-1)
out = torch.matmul(attn_weights, v)

In [17]:
torch.sparse.softmax(sp_attn_weights, dim=-1)

NotImplementedError: Could not run 'aten::_sparse_softmax' with arguments from the 'SparseCsrCPU' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'aten::_sparse_softmax' is only available for these backends: [MPS, Meta, SparseCPU, SparseMPS, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradHIP, AutogradXLA, AutogradMPS, AutogradIPU, AutogradXPU, AutogradHPU, AutogradVE, AutogradLazy, AutogradMTIA, AutogradMAIA, AutogradPrivateUse1, AutogradPrivateUse2, AutogradPrivateUse3, AutogradMeta, AutogradNestedTensor, Tracer, AutocastCPU, AutocastMTIA, AutocastMAIA, AutocastXPU, AutocastMPS, AutocastCUDA, FuncTorchBatched, BatchedNestedTensor,

In [16]:
(sp_attn_weights - sp_attn_weights.max()).exp()

NotImplementedError: Could not run 'aten::max' with arguments from the 'SparseCsrCPU' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'aten::max' is only available for these backends: [CPU, MPS, Meta, QuantizedCPU, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradHIP, AutogradXLA, AutogradMPS, AutogradIPU, AutogradXPU, AutogradHPU, AutogradVE, AutogradLazy, AutogradMTIA, AutogradMAIA, AutogradPrivateUse1, AutogradPrivateUse2, AutogradPrivateUse3, AutogradMeta, AutogradNestedTensor, Tracer, AutocastCPU, AutocastMTIA, AutocastMAIA, AutocastXPU, AutocastMPS, AutocastCUDA, FuncTorchBatched, BatchedNestedTensor, FuncTorchVmapMode, Batched